<h2>Setting Stuff Up</h2>

Here we import some packages that we'll need in various places.

In [1]:
!mkdir -p ~/agave

%cd ~/agave

!pip3 install --upgrade setvar

import re
import os
import sys
from setvar import *
from time import sleep

# This cell enables inline plotting in the notebook
%matplotlib inline

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

/home/jovyan/agave
Requirement already up-to-date: setvar in /opt/conda/lib/python3.6/site-packages
You are using pip version 9.0.1, however version 10.0.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


<h2>Set Variables</h2>

The setvar() package gives us a bash-like way of setting environment variables in Python. It's a lot easier to type `PORT=10022` than it is to type `os.environ["PORT"]="10022"`. The setvar package also supports comments and running a command in the background and getting its output.

In [19]:
setvar("""
AGAVE_USERNAME=dooley
MACHINE_IP=$VM_IPADDRESS
MACHINE_NAME=jupyter
DOMAIN=agaveplatform.org
EMAIL=deardooley@gmail.com
MACHINE_USERNAME=jovyan
PORT=10022

DOCKERHUB_NAME=dooley
WORK_DIR=/home/${MACHINE_USERNAME}
HOME_DIR=/home/${MACHINE_USERNAME}
SCRATCH_DIR=/home/${MACHINE_USERNAME}
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq
AGAVE_TENANTS_API_BASEURL=https://public.agaveapi.co/tenants
APP_NAME=funwave-tvd-${MACHINE_NAME}-${AGAVE_USERNAME}
STORAGE_MACHINE=${MACHINE_NAME}-storage-${AGAVE_USERNAME}
EXEC_MACHINE=${MACHINE_NAME}-exec-${AGAVE_USERNAME}
REQUESTBIN_URL=$(requestbin-create)
""")

AGAVE_USERNAME=dooley
MACHINE_IP=204.90.47.30
MACHINE_NAME=jupyter
DOMAIN=agaveplatform.org
EMAIL=deardooley@gmail.com
MACHINE_USERNAME=jovyan
PORT=10022
DOCKERHUB_NAME=dooley
WORK_DIR=/home/jovyan
HOME_DIR=/home/jovyan
SCRATCH_DIR=/home/jovyan
DEPLOYMENT_PATH=agave-deployment
AGAVE_JSON_PARSER=jq
AGAVE_TENANTS_API_BASEURL=https://public.agaveapi.co/tenants
APP_NAME=funwave-tvd-jupyter-dooley
STORAGE_MACHINE=jupyter-storage-dooley
EXEC_MACHINE=jupyter-exec-dooley
REQUESTBIN_URL=https://requestbin.agaveapi.co/1nf2h871


If you are running locally using Docker Compose, you will need to pull the ip and port of your reverse tunnel from the sandbox. Uncomment the following command, and enter below

In [20]:
if os.environ.get('USE_TUNNEL') == "True" :
    !ssh -o StrictHostKeyChecking=no -o UserKnownHostsFile=/dev/null sandbox 'curl -s  http://localhost:4040/api/tunnels | jq -r '.tunnels[0].public_url'' > ngrok_url.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\1#' > ngrok_host.txt
    !cat ngrok_url.txt | sed 's|^tcp://||' | sed -r 's#(.*):(.*)#\2#'  > ngrok_port.txt

    setvar("""
    MACHINE_IP=$(cat ngrok_host.txt)
    PORT=$(cat ngrok_port.txt)
    """)

MACHINE_IP=0.tcp.ngrok.io
PORT=12873


<h2>PushBullet</h2>
If you wish to use pushbullet to receive status updates, then run the cell below and enter your pushbullet token.

In [21]:
readpass("PBTOK")

Password or secret: PBTOK
Reading file `PBTOK.txt'


<h2>Agave Password</h2>

We'll be running several commands that require the Agave password. We'll store it here for convenience.

In [22]:
if not os.environ.get('AGAVE_PASSWORD') :
    readpass("AGAVE_PASSWORD")

<h2>Setting up Agave</h2>
Agave uses machines called "tenants" to manage user login and authentication. In this step, we tell Agave we are using the standard proction tenant.

In [23]:
!tenants-init -t agave.prod

You are now configured to interact with the APIs at https://public.agaveapi.co/


<h3>Create the Client</h3>

In this next step we delete the client if it exists. Chances are, yours doesn't yet. We put this command here in case, for some reason, you want to re-create your client later on. If you delete the client you intend to create before you create it, no harm is done.

In [24]:
!clients-delete $APP_NAME

Successfully deleted client funwave-tvd-jupyter-dooley


In this step we create the client. Clients provide a way of encapsulating resources connected to a single project. Through the client, you will receive a token which you can use to run most of the Agave commands.

In [ ]:
!clients-create -S -N $APP_NAME

Create the token for your client. You will, from this point on, use this token to run the remainder of the Agave commands in this tutorial.

In [ ]:
!auth-tokens-create 